In [1]:
# Instal dependency
#!pip install open_clip_torch

In [2]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import re

c:\Users\Dung\anaconda3\envs\aic2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parse data path

In [3]:
keyframes_dir = r'D:\VN_Multi_User_Video_Search\frontend\ai\public\data\Keyframes'
all_keyframe_paths = dict()
pattern = re.compile(r'^[A-Z]\d+_V\d+$')

for folder_name in os.listdir(keyframes_dir):
    if pattern.match(folder_name):
        folder_path = os.path.join(keyframes_dir, folder_name)
        if os.path.isdir(folder_path):
            data_part, video_id = folder_name.split('_')
            keyframe_paths = sorted(glob.glob(os.path.join(folder_path, '*.jpg')))
            if keyframe_paths:
                all_keyframe_paths.setdefault(data_part, {})[video_id] = keyframe_paths

# Model

In [6]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_text_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_basic_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_s13m_b4k'),
 ('ViT-

In [ ]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cuda


In [5]:
bs = 4
save_dir = './clip-features-32'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)


for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    out_key_dir = os.path.join(save_dir, key)
    os.makedirs(out_key_dir, exist_ok=True)

    for video_id in tqdm(sorted(video_keyframe_paths.keys()), leave=False):
        out_path = os.path.join(out_key_dir, f"{video_id}.npy")
        if os.path.exists(out_path):        # >>> BỎ QUA nếu đã có kết quả
            continue

        video_feats = []
        frames = video_keyframe_paths[video_id]
        for i in range(0, len(frames), bs):
            batch_paths = frames[i:i+bs]
            imgs = [preprocess(Image.open(p)).unsqueeze(0) for p in batch_paths]
            imgs = torch.cat(imgs, dim=0).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=="cuda")):
                feats = model.encode_image(imgs)
                feats = (feats / feats.norm(dim=-1, keepdim=True)).detach().cpu().numpy().astype(np.float32)

            video_feats.extend([f for f in feats])

        np.save(out_path, video_feats)

  0%|          | 0/30 [00:00<?, ?it/s]C:\Users\Dung\AppData\Local\Temp\ipykernel_10672\3088525276.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=="cuda")):
100%|██████████| 30/30 [1:51:14<00:00, 222.48s/it]
